In [6]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader
import pandas as pd
from PIL import Image
import match_id_to_image as id2img
import os
import torch


In [7]:
# Step 1: Load weighted ResNet-50 Model
weighted_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Step 2: Modify the Last Layer
num_classes = 2
num_ftrs = weighted_model.fc.in_features
weighted_model.fc = nn.Linear(num_ftrs, num_classes)

# Step 3: Prepare Your Dataset
# Get the directory of the current script
current_dir = r'C:\Users\andre\dev\BSc-Project\BSc-Project\data'
# Construct relative paths based on the current directory to csv's
csv_dir = os.path.join(current_dir, '..', 'data/splitted_csv/')
# Construct relative paths based on the current directory to data's
data_dir = os.path.join(current_dir, '..', 'pad-ufes/images/')

# Example transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load CSV files
train_csv = os.path.join(csv_dir, 'm_f_ca_nc_train_0_0.00_0.csv')
val_csv = os.path.join(csv_dir, 'm_f_ca_nc_val_0_0.00_0.csv')
test_csv = os.path.join(csv_dir, 'm_f_ca_nc_test_0.csv')


# Create datasets
train_dataset = id2img.CustomDataset(csv_file=train_csv, data_dir=data_dir, transform=transform)
val_dataset = id2img.CustomDataset(csv_file=val_csv, data_dir=data_dir, transform=transform)
test_dataset = id2img.CustomDataset(csv_file=test_csv, data_dir=data_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Step 5: Define Loss Function and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(weighted_model.parameters(), lr=0.001, momentum=0.9)

# Step 6: Train the Model
num_epochs = 1
for epoch in range(num_epochs):
    weighted_model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = weighted_model(inputs)
        # Apply sigmoid activation to convert logits to probabilities
        probabilities = torch.sigmoid(outputs)
        # Convert labels to one-hot encoding
        labels_one_hot = torch.zeros(labels.size(0), num_classes)
        labels_one_hot.scatter_(1, labels.unsqueeze(1), 1)  # Fill one-hot tensor
        # Calculate the loss
        loss = criterion(outputs, labels_one_hot)
        loss.backward()
        optimizer.step()

# Step 7: Evaluate the Model
weighted_model.eval()

def evaluate_model(model, dataloader):
    model.eval()  # Set model to evaluation mode
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()
    predicted = predicted        
    accuracy = total_correct / total_samples
    print(predicted)
    
    return predicted

# Evaluate on validation set
val_accuracy = evaluate_model(weighted_model, val_loader)
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Evaluate on test set
test_accuracy = evaluate_model(weighted_model, test_loader)
print(f'Test Accuracy: {test_accuracy:.4f}')


tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1])


TypeError: unsupported format string passed to tuple.__format__